In [6]:
import pm4py
from pm4py.objects.ocel.obj import OCEL
from typing import Optional
import pandas as pd

In [8]:
ocel = pm4py.read_ocel2_sqlite("../data/event_logs/socel_hinge.sqlite")

/home/grkmr/.cache/pypoetry/virtualenvs/ocean-backend-6RZTwyJ5-py3.10/lib/python3.10/site-packages/pm4py/objects/ocel/importer/sqlite/variants/ocel20.py:70: UserWarning: There are unsatisfied OCEL 2.0 constraints in the given relational database: ['const_12_existence_type_ocel_time_obj_type_spec_tables', 'const_14_primary_key_object_event_map_type_tables', 'const_15_primary_key_object_event_tables', 'const_16_primary_key_event_object_table', 'const_17_primary_key_object_object_table', 'const_18_primary_key_event_type_spec_tables', 'const_19_foreign_key_event', 'const_20_foreign_key_object', 'const_21_foreign_key_event_object', 'const_22_foreign_key_object_object', 'const_23_foreign_key_event_type_specific', 'const_24_foreign_key_object_type_specific']
  warnings.warn("There are unsatisfied OCEL 2.0 constraints in the given relational database: "+str(unsatisfied))
/home/grkmr/.cache/pypoetry/virtualenvs/ocean-backend-6RZTwyJ5-py3.10/lib/python3.10/site-packages/pm4py/objects/log/util/da

# Filter

In [11]:
from typing import Literal, Optional
from pydantic import BaseModel

class BaseFilterConfig(BaseModel):
    type: str
    mode: Optional[Literal["include", "exclude"]] = "include"

## Filter Entities by type

In [12]:
def filter_by_entity_types(entity_table:pd.DataFrame,entity_type_field:str,entity_type_list:list[str],include:bool=True):
    eventMask = entity_table[entity_type_field].isin(entity_type_list)
    return eventMask if include else ~eventMask

### Filter by activity name

In [13]:
class EventTypeFilterConfig(BaseFilterConfig):
    type: Literal["event_type"]
    event_types: list[str]

In [14]:
def filter_by_event_types(ocel:OCEL,event_types:list[str],include:bool=True):
    return filter_by_entity_types(ocel.events, ocel.event_activity, event_types, include)

### Filter by object type

In [15]:
class ObjectTypeFilterConfig(BaseFilterConfig):
    type:Literal["object_type"]
    object_types:list[str]

In [15]:
def filter_by_object_types(ocel:OCEL,object_types:list[str],include:bool=True):
    return filter_by_entity_types(ocel.objects, ocel.object_id_column, object_types, include)

## Filter Entities by frequencies

In [17]:
def filter_by_entity_frequency(entity_tabele:pd.DataFrame, entity_type_field:str, threshold:int):
    cumulative_freq = entity_table.value_counts(entity_type_field, normalize=True).cumsum()
    included_entities = cumulative_freq[cumulative_freq <= threshold].index
    return filter_by_entity_types(entity_table,entity_type_field,included_entities)

### Filter by Activity type frequency 

In [19]:
def filter_by_activity_frequency(ocel:OCEL, threshold:int):
    return filter_by_entity_frequency(ocel.events, ocel.event_activity, threshold)

### Filter by Object type frequency

In [21]:
def filter_by_object_frequency(ocel:OCEL, threshold:int):
    return filter_by_entity_frequency(ocel.objects, ocel.object_id_column, threshold)

## Filter by time range

In [79]:
def filter_by_time_range(ocel:OCEL,start_time:Optional[str]=None, end_time:Optional[str]=None , include:bool=True):
    if start_time is not None:
        start_time = pd.Timestamp(start_time, tz="UTC")
    if end_time is not None:
        end_time = pd.Timestamp(end_time, tz="UTC")
    
    events_df = ocel.events

    mask = pd.Series([True] * len(events_df), index=events_df.index)
    if start_time is not None:
        mask &= events_df["ocel:timestamp"] >= start_time
    if end_time is not None:
        mask &= events_df["ocel:timestamp"] <= end_time

    return mask if include else ~mask